In [1]:
import pickle
import os

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
os.environ["AWS_PROFILE"] = "default"

In [3]:
import subprocess
subprocess.Popen(["mlflow", "server", 
"--backend-store-uri", "sqlite:///mlflow.db", 
"--default-artifact-root", "s3://mlopszoomcamp-alex"]) 

<Popen: returncode: None args: ['mlflow', 'server', '--backend-store-uri', '...>

2023/06/16 05:24:59 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/06/16 05:24:59 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [4]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("green-taxi-duration")

2023/06/16 05:25:14 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlopszoomcamp-alex/1', creation_time=1686893115096, experiment_id='1', last_update_time=1686893115096, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


/home/codespace/.local/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [8]:
from mlflow.tracking import MlflowClient

RUN_ID = 'a4b217a84e3a44ad870271b75331eb6c'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [11]:
path = client.download_artifacts(run_id=RUN_ID, path='model/model.pkl')

/tmp/ipykernel_2221/4213840234.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID, path='model/model.pkl')


In [14]:
with open(path, 'rb') as f_out:
    downloaded_model = pickle.load(f_out)

In [15]:
downloaded_model

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, min_samples_leaf=10,
                                       n_jobs=-1, random_state=0))])

In [16]:
path

'/tmp/tmph65jft74/model/model.pkl'